# Imports & Contants
---

In [68]:
# Native
import os
import csv
from urllib.request import urlopen as uOpen, Request as uReq
from IPython.display import Image

# Third-parties
from bs4 import BeautifulSoup as soup

In [3]:
BASE_URL  = 'https://bulbapedia.bulbagarden.net'
END_URL   = '/wiki/%3F%3F%3F_(Pok%C3%A9mon)'
CSV_NAME  = 'pokemon.csv'
CURRENT_KEYS = [
    'image_link', 
    'index', 
    'name', 
    'categories', 
    'jp_name', 
    'jp_rom_name', 
    'types', 
    'abilities', 
    'gender_ratio', 
    'catch_rate', 
    'egg_groups', 
    'hatch_time', 
    'height', 
    'weight', 
    'mega_stones',
    'evs'
]

# Util Functions
---

- ### Formatar uma saída.

In [132]:
def print_formatted_message(poke_info):
    poke_image = Image(url=poke_info['image_link'])
    category_label   = 'Categories'  if len(poke_info['categories'])  > 1 else 'Category'
    type_label       = 'Types'       if len(poke_info['types'])       > 1 else 'Type'
    ability_label    = 'Abilities'   if len(poke_info['abilities'])   > 1 else 'Ability'
    egg_group_label  = 'Egg Groups'  if len(poke_info['egg_groups'])  > 1 else 'Egg Group'
    mega_stone_label = 'Mega Stones' if len(poke_info['mega_stones']) > 1 else 'Mega Stone'
    
    evs = ''
    if len(poke_info['evs']) == 6:
        evs += '\n - '
        evs += '\n - '.join(poke_info['evs'])
    else:
        for evc in poke_info['evs']:
            evs += f'\n - {evc[0]}:\n\t- '
            evs += '\n\t- '.join(evc[1])
            

    print('########################################################################################')
    display(poke_image)
    print("""\
Image: %s
Index: %d
Name: %s / %s (%s)
%s: %s
%s: %s
============================================================
%s: \n - %s
============================================================
Gender Ratio: %s
Catch Rate: %s
============================================================
%s: %s
Hatch Time: %s
============================================================
Height: %s
Weight: %s
============================================================
%s: %s
============================================================
EVs: %s
########################################################################################\n\n
    """ % (
        poke_info['image_link'], poke_info['index'], poke_info['name'], 
        poke_info['jp_name'], poke_info['jp_rom_name'],
        
        category_label, ' / '.join(poke_info['categories']),
        type_label,     ' / '.join(poke_info['types']),
        ability_label,  ' \n - '.join(poke_info['abilities']),
        
        ' / '.join(poke_info['gender_ratio']) if len(poke_info['gender_ratio']) > 0 else "Genderless",
        
        poke_info['catch_rate'],
        
        egg_group_label, ' / '.join(poke_info['egg_groups']),
        poke_info['hatch_time'],
        
        ' / '.join(poke_info['height']),
        ' / '.join(poke_info['weight']),
        
        mega_stone_label,
        ' / '.join(poke_info['mega_stones']) if len(poke_info['mega_stones']) > 0 else '---',
        
        evs
    ))


---
&nbsp;
- ### Recriar arquivos dado o nome.

In [5]:
def recreate_file(file_name):
    if (os.path.exists(file_name)):
        os.remove(file_name)

    open(file_name, 'x')


- ### Escrever headers e colunas csv

In [6]:
def write_csv_headers(file_name):
    with open(file_name, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=CURRENT_KEYS)
        writer.writeheader()

- ### Format Text to CSV

In [137]:
def format_to_csv(poke_info):
    evs = ''
    if len(poke_info['evs']) == 6:
        evs += ' - '
        evs += '\n - '.join(poke_info['evs'])
    else:
        for evc in poke_info['evs']:
            evs += f' - {evc[0]}:\n\t- '
            evs += '\n\t- '.join(evc[1])
            evs += '\n'
            
    return {
        'image_link':   f'=IMAGE(\"{poke_info["image_link"]}\")',
        'index':        poke_info['index'],
        'name':         poke_info['name'],
        'categories':   ' \n'.join(['- ' + p for p in poke_info['categories']]),
        'jp_name':      poke_info['jp_name'],
        'jp_rom_name':  poke_info['jp_rom_name'],
        'types':        ' \n'.join(['- ' + p for p in poke_info['types']]),
        'abilities':    ' \n'.join(['- ' + p for p in poke_info['abilities']]),
        'gender_ratio': ' \n'.join(['- ' + p for p in poke_info['gender_ratio']]),
        'catch_rate':   poke_info['catch_rate'],
        'egg_groups':   ' \n'.join(['- ' + p for p in poke_info['egg_groups']]),
        'hatch_time':   poke_info['hatch_time'],
        'height':       ' \n'.join(['- ' + p for p in poke_info['height']]),
        'weight':       ' \n'.join(['- ' + p for p in poke_info['weight']]),
        'mega_stones':  ' \n'.join(['- ' + p for p in poke_info['mega_stones']]),
        'evs':          evs
    }
        

- ### Append to CSV File

In [8]:
def append_csv(poke_info, file_name):
    formatted_poke = format_to_csv(poke_info)
    keys           = formatted_poke.keys()

    with open(file_name, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writerow(formatted_poke)
        

- ### Pegar link de um pokémon

In [9]:
def get_pokemon_link(poke_name):
        return f'/wiki/{poke_name.capitalize()}_%28Pok%C3%A9mon%29'

- ### Pegar link da poke_table pelo titulo

In [10]:
def get_link_by_title(info_table, title):
    return info_table.find('a', attrs={'title': title})

In [11]:
def get_all_links_by_title(info_table, title):
    return info_table.find_all('a', attrs={'title': title})

# Core Functions
---

- ### Carregar a página destino e retornar o soup.

In [31]:
def get_poke_soup(link):
    uClient         = uReq(link, headers={'User-Agent': 'Magic Browser'})
    uCon            = uOpen(uClient, None, 5)
    poke_page_html  = uCon.read()
    uCon.close()
    
    return soup(poke_page_html, 'html.parser')

---
&nbsp;
- ### Checar se é a última página a ser verificada.
    - Em alguns casos, quando tá perto do lançamento de um novo jogo e só se tem informação de um ou mais pokémons, às vezes o Bulbapédia coloca informações não oficiais com um layout diferente.

In [13]:
def check_last_page(poke_soup):
    return poke_soup.find(id='mw-content-text').table.a['href'] == "/wiki/File:BulbaShadow.png"

---
&nbsp;
- ### Pegar informações do Pokémon
    - Essa é a função core, todas as funções de extração de dados são chamadas aqui.

In [14]:
def get_poke_info(poke_soup):
    info_table = poke_soup                      \
                    .find(id='mw-content-text') \
                    .find_all('table', recursive=False)[1]

    core          = get_core_poke_info(info_table)
    types         = get_poke_types(info_table)
    abilities     = get_poke_abilities(info_table)
    gender_ratios = get_poke_gender_ratio(info_table)
    catch_rate    = get_poke_catch_ratio(info_table)
    egg_groups    = get_poke_egg_group(info_table)
    hatch_time    = get_poke_hatch_time(info_table)
    height        = get_poke_height(info_table)
    weight        = get_poke_weight(info_table)
    mega_stones   = get_poke_megastones(info_table)
    evs           = get_poke_evs(info_table)
    
    return {
        'image_link':   core[0],
        'index':        core[1],
        'name':         core[2],
        'categories':   core[3],
        'jp_name':      core[4],
        'jp_rom_name':  core[5],
        'types':        types,
        'abilities':    abilities,
        'gender_ratio': gender_ratios,
        'catch_rate':   catch_rate,
        'egg_groups':   egg_groups,
        'hatch_time':   hatch_time,
        'height':       height,
        'weight':       weight,
        'mega_stones':  mega_stones,
        'evs':          evs
    }

# Data Extraction Functions
---

- ### Pegar o link do próximo pokémon
    - Nessa parte da página, o Bulbapédia pode colocar trs a mais pra informar algo (geralmente relacionado ao Pokemon Go), por isso esse tratamento.

In [15]:
def get_next_pokemon_link(poke_soup):
    npl = poke_soup.find(id='mw-content-text').table
    try:
        npl = npl.findChildren('tr', recursive=False)[1]    \
                 .findChildren('td', recursive=False)[2]
    except IndexError:
        npl = npl.findChildren('tr', recursive=False)[0]    \
                 .findChildren('td', recursive=False)[2]
    finally:
        return npl.find('a')['href']            

---
&nbsp;
- ### Pegar informações core do pokémon:
    - Index (número na NationalDex)
    - Nome
    - Categoria
    - Nome (JP)
    - Nome (JP/Romaji)

In [16]:
def get_core_poke_info(info_table):
    base_info_container = info_table.tr.td.table.tr
    info_container      = base_info_container.td.table.tr      \
                            .find_all('td', recursive=False)
    
    category   = info_container[0].a.find_all('span')
    categories = [cat.text for cat in category]
    
    if len(categories) > 1:
        categories = [f'{cat} Pokémon' for cat in categories if cat.find('Pokémon') == -1]
    
    image_link = 'https:' + info_table.img['src']
    
    return (
        image_link,                                                  # image_link
        int(base_info_container.th.find('a').text.replace('#', '')), # index
        info_container[0].big.text,                                  # name
        categories,                                                  # category
        info_container[1].span.text,                                 # jp_name
        info_container[1].i.text                                     # jp_rom_name
    )

---
&nbsp;
- ### Pegar tipos do pokémon.

In [17]:
def get_poke_types(info_table):
    types = info_table.find_all('tr', recursive=False)[1]          \
                      .table.find('td', attrs={'style': None})     \
                      .find_all('a')
    
    return [t.text for t in types if t.text != 'Unknown']


---
&nbsp;
- ### Pegar abilidades do pokémon.

In [18]:
def get_poke_abilities(info_table):
    ability_title = get_link_by_title(info_table, 'Ability')
    abilities     = []

    for parent in ability_title.parents:        
        if parent.name == 'td':
            abilities_container = parent.table.find_all('td')
            
            for td in abilities_container:
                if not td.has_attr('style') or (not 'display: none' in td['style']):
                    hidden_ability_container = td.find('small')
                    ability                  = td.find('a').text
                    
                    if  hidden_ability_container != None:
                        ability += f' ({hidden_ability_container.text.strip()})'

                    abilities.append(ability)
            break
    
    
    return abilities

- ### Pegar gender ratios.

In [19]:
def get_poke_gender_ratio(info_table):
    gender_ratio_titles = get_link_by_title(info_table, 'List of Pokémon by gender ratio')
    
    for parent in gender_ratio_titles.parents:        
        if parent.name == 'td':
            ratios_container = parent.table.find_all('td')
            
            for td in ratios_container:
                unformatted_ratio = td.find('a')
                
                if not td.has_attr('style') and unformatted_ratio != None:
                    return [r.strip() for r in td.find('a').text.split(',')]

            break
            
    return []

- ### Pegar catch rate

In [20]:
def get_poke_catch_ratio(info_table):
    catch_ratio_titles = get_link_by_title(info_table, 'Catch rate')
    
    for parent in catch_ratio_titles.parents:        
        if parent.name == 'td':
            return parent.table.td.text.strip()
            
    return ''

- ### Pegar Egg Group.

In [21]:
def get_poke_egg_group(info_table):
    egg_group_link = get_link_by_title(info_table, 'Egg Group')
    
    for parent in egg_group_link.parents:
        if parent.name == 'td':
            egg_groups = parent.table.find_all('span')
            return [eg.text for eg in egg_groups]
    
    return []

- ### Pegar Hatch Time

In [22]:
def get_poke_hatch_time(info_table):
    hatch_time_link = get_link_by_title(info_table, 'Egg cycle')

    for parent in hatch_time_link.parents:
        if parent.name == 'td':
            hatch_time           = parent.table.td.text.strip()
            hatch_time_formatted = hatch_time.split('Egg')
            
            if len(hatch_time_formatted) > 1:
                hatch_time =  f'{hatch_time_formatted[0]} (Egg not obtainable)'
                
            return hatch_time
    
    return ''

- ### Pegar Altura

In [23]:
def get_poke_height(info_table):
    height_link = get_link_by_title(info_table, 'List of Pokémon by height')
    
    for parent in height_link.parents:
        if parent.name == 'td':
            height_container = parent.table.find('tr', attrs={'style': None})
            return [h.text.strip() for h in height_container.find_all('td')]
    
    return []

- ### Pegar Peso

In [24]:
def get_poke_weight(info_table):
    weight_link = get_link_by_title(info_table, 'Weight')
    
    for parent in weight_link.parents:
        if parent.name == 'td':
            weight_container = parent.table.find('tr', attrs={'style': None})
            return [w.text.strip() for w in weight_container.find_all('td')]
    
    return []

- ### MegaStone

In [25]:
def get_poke_megastones(info_table):
    mega_stone = get_all_links_by_title(info_table, 'Mega Stone')
    mega_stone = [ms for ms in mega_stone if ms['href'] != '/wiki/Mega_Stone']
    
    return [ms.text for ms in mega_stone]

- ### EVs

In [88]:
def get_poke_evs(info_table):
    evs_link       = get_link_by_title(info_table, 'List of Pokémon by effort value yield')
    
    for parent in evs_link.parents:
        if parent.name == 'td':
            evs_container   = parent.table.find_all('tr', attrs={'style': None})
            evs_labels      = [evc.text.strip() for evc in evs_container if len(evc.find_all('td')) == 1 and not 'Total' in evc.find('td').text]
            evs_container  = [evc for evc in evs_container if len(evc.find_all('td')) > 1]

            if len(evs_labels) > 0:
                total_evs = []
                
                for index, label in enumerate(evs_labels):
                    total_evs.append((label, get_evs(evs_container[index])))
                
                return total_evs
            else:
                return get_evs(evs_container[0])
    
    return []

In [61]:
def get_evs(evs_container):
    evs_label_list = ['HP', 'Atk', 'Def', 'Sp.Atk', 'Sp.Def', 'Speed']
    evs_unformatted = evs_container.find_all('td')
    evs             = []

    for index, ev in enumerate(evs_unformatted):
        selec_label = evs_label_list[index]
        ev_text     = ev.text.split(selec_label)[0]

        evs.append(f'{selec_label}: {ev_text.strip()}'.strip())

    return evs
    

# Main Cycle
---

In [138]:
recreate_file(CSV_NAME)
write_csv_headers(CSV_NAME)

next_pokemon_link = get_pokemon_link('Bulbasaur')
tries             = 10
while next_pokemon_link != END_URL:
#     try:
    poke_soup = get_poke_soup(f'{BASE_URL}{next_pokemon_link}')
#     except:
#         if tries > 0:
#             tries -= 1
#             print()
#             print()
#             print('*****************************************')
#             print('* Something went wrong, trying again... *')
#             print('*****************************************')
#             print()
#             print()
#             continue
#         break
            
    
    if check_last_page(poke_soup):
        break

    poke_info = get_poke_info(poke_soup)

    append_csv(poke_info, CSV_NAME)
    print(next_pokemon_link)
    print_formatted_message(poke_info)

    next_pokemon_link = get_next_pokemon_link(poke_soup)
    tries = 10

print("\n\n########## FINISHED ##########\n\n")

/wiki/Bulbasaur_%28Pok%C3%A9mon%29
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/2/21/001Bulbasaur.png/250px-001Bulbasaur.png
Index: 1
Name: Bulbasaur / フシギダネ (Fushigidane)
Category: Seed Pokémon
Types: Grass / Poison
Abilities: 
 - Overgrow 
 - Chlorophyll (Hidden Ability)
Gender Ratio: 87.5% male / 12.5% female
Catch Rate: 45 (11.9%)
Egg Groups: Monster / Grass
Hatch Time: 5140 - 5396 steps
Height: 2'04" / 0.7 m
Weight: 15.2 lbs. / 6.9 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 0
 - Def: 0
 - Sp.Atk: 1
 - Sp.Def: 0
 - Speed: 0
########################################################################################


    
/wiki/Ivysaur_(Pok%C3%A9mon)
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/7/73/002Ivysaur.png/250px-002Ivysaur.png
Index: 2
Name: Ivysaur / フシギソウ (Fushigisou)
Category: Seed Pokémon
Types: Grass / Poison
Abilities: 
 - Overgrow 
 - Chlorophyll (Hidden Ability)
Gender Ratio: 87.5% male / 12.5% female
Catch Rate: 45 (11.9%)
Egg Groups: Monster / Grass
Hatch Time: 5140 - 5396 steps
Height: 3'03" / 1.0 m
Weight: 28.7 lbs. / 13.0 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 0
 - Def: 0
 - Sp.Atk: 1
 - Sp.Def: 1
 - Speed: 0
########################################################################################


    
/wiki/Venusaur_(Pok%C3%A9mon)
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/a/ae/003Venusaur.png/250px-003Venusaur.png
Index: 3
Name: Venusaur / フシギバナ (Fushigibana)
Category: Seed Pokémon
Types: Grass / Poison
Abilities: 
 - Overgrow (Venusaur) 
 - Chlorophyll (Hidden Ability) 
 - Thick Fat (Mega Venusaur)
Gender Ratio: 87.5% male / 12.5% female
Catch Rate: 45 (11.9%)
Egg Groups: Monster / Grass
Hatch Time: 5140 - 5396 steps
Height: 6'07" / 2.0 m
Weight: 220.5 lbs. / 100.0 kg
Mega Stone: Venusaurite
EVs: 
 - HP: 0
 - Atk: 0
 - Def: 0
 - Sp.Atk: 2
 - Sp.Def: 1
 - Speed: 0
########################################################################################


    
/wiki/Charmander_(Pok%C3%A9mon)
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/7/73/004Charmander.png/250px-004Charmander.png
Index: 4
Name: Charmander / ヒトカゲ (Hitokage)
Category: Lizard Pokémon
Type: Fire
Abilities: 
 - Blaze 
 - Solar Power (Hidden Ability)
Gender Ratio: 87.5% male / 12.5% female
Catch Rate: 45 (11.9%)
Egg Groups: Monster / Dragon
Hatch Time: 5140 - 5396 steps
Height: 2'00" / 0.6 m
Weight: 18.7 lbs. / 8.5 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 0
 - Def: 0
 - Sp.Atk: 0
 - Sp.Def: 0
 - Speed: 1
########################################################################################


    
/wiki/Charmeleon_(Pok%C3%A9mon)
########################################################################################


...

Image: https://cdn.bulbagarden.net/upload/thumb/2/27/805Stakataka.png/250px-805Stakataka.png
Index: 805
Name: Stakataka / ツンデツンデ (Tundetunde)
Category: Rampart Pokémon
Types: Rock / Steel
Ability: 
 - Beast Boost
Gender Ratio: Genderless
Catch Rate: 30 (8.8%)
Egg Group: Undiscovered
Hatch Time: 30840 - 31096 steps (Egg not obtainable)
Height: 18'01" / 5.5 m
Weight: 1807.8 lbs. / 820.0 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 0
 - Def: 3
 - Sp.Atk: 0
 - Sp.Def: 0
 - Speed: 0
########################################################################################


    
/wiki/Blacephalon_(Pok%C3%A9mon)
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/a/a5/806Blacephalon.png/250px-806Blacephalon.png
Index: 806
Name: Blacephalon / ズガドーン (Zugadoon)
Category: Fireworks Pokémon
Types: Fire / Ghost
Ability: 
 - Beast Boost
Gender Ratio: Genderless
Catch Rate: 30 (8.8%)
Egg Group: Undiscovered
Hatch Time: 30840 - 31096 steps (Egg not obtainable)
Height: 5'11" / 1.8 m
Weight: 28.7 lbs. / 13.0 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 0
 - Def: 0
 - Sp.Atk: 3
 - Sp.Def: 0
 - Speed: 0
########################################################################################


    
/wiki/Zeraora_(Pok%C3%A9mon)
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/a/a7/807Zeraora.png/250px-807Zeraora.png
Index: 807
Name: Zeraora / ゼラオラ (Zeraora)
Category: Thunderclap Pokémon
Type: Electric
Ability: 
 - Volt Absorb
Gender Ratio: Genderless
Catch Rate: 3 (1.6%)
Egg Group: Undiscovered
Hatch Time: 30840 - 31096 steps (Egg not obtainable)
Height: 4'11" / 1.5 m
Weight: 98.1 lbs. / 44.5 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 0
 - Def: 0
 - Sp.Atk: 0
 - Sp.Def: 0
 - Speed: 3
########################################################################################


    
/wiki/Meltan_(Pok%C3%A9mon)
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/d/d1/808Meltan.png/250px-808Meltan.png
Index: 808
Name: Meltan / メルタン (Meltan)
Category: Hex Nut Pokémon
Type: Steel
Ability: 
 - Magnet Pull
Gender Ratio: Genderless
Catch Rate: 3 (1.6%)
Egg Group: Undiscovered
Hatch Time: 30840 - 31096 steps (Egg not obtainable)
Height: 0'08" / 0.2 m
Weight: 17.6 lbs. / 8.0 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 1
 - Def: 0
 - Sp.Atk: 0
 - Sp.Def: 0
 - Speed: 0
########################################################################################


    
/wiki/Melmetal_(Pok%C3%A9mon)
########################################################################################


Image: https://cdn.bulbagarden.net/upload/thumb/e/ea/809Melmetal.png/250px-809Melmetal.png
Index: 809
Name: Melmetal / メルメタル (Melmetal)
Category: Hex Nut Pokémon
Type: Steel
Ability: 
 - Iron Fist
Gender Ratio: Genderless
Catch Rate: 3 (1.6%)
Egg Group: Undiscovered
Hatch Time: 30840 - 31096 steps (Egg not obtainable)
Height: 8'02" / 2.5 m
Weight: 1763.7 lbs. / 800.0 kg
Mega Stone: ---
EVs: 
 - HP: 0
 - Atk: 3
 - Def: 0
 - Sp.Atk: 0
 - Sp.Def: 0
 - Speed: 0
########################################################################################


    


########## FINISHED ##########




# Teste
---

In [73]:
def tst_pok(poke_name, backup=False):
    next_pokemon_link = get_pokemon_link(poke_name)
    poke_soup         = get_poke_soup(f'{BASE_URL}{next_pokemon_link}')
    poke_info         = get_poke_info(poke_soup)
    
    if backup:
        with open('a.html', 'w') as f:
            f.write(poke_soup.prettify())

    print_formatted_message(poke_info)